#### Este script organiza la información para el documento de estaciones Bicimad, y se incluyen 5 columnas nuevas con los identificadores para las estaciones de meteorología correspondientes a 3 de temperatura y 2 de precipitación 

In [3]:
import os
import json
import pandas as pd
from geopy.distance import geodesic

In [4]:
# Ruta de la carpeta que contiene los archivos
folder_path = 'datos/'

# Read the CSV file into a DataFrame, specifying ';' as the separator
file_path1 = os.path.join(folder_path, 'bases_bicimad.xlsx')
df_estaciones_bicimad = pd.read_excel(file_path1)
file_path2 = os.path.join(folder_path, 'Estaciones_control_datos_meteorologicos.csv')
df_estaciones_meteo = pd.read_csv(file_path2, sep=';')


In [7]:
# Lista de columnas a eliminar (usando los nombres exactos de tu Index)
columnas_a_eliminar = [
    "Gis_X",
    "Gis_Y",
    "Calle",
    "Nº Finca"
]

try:
    # Intenta eliminar las columnas
    df_estaciones_bicimad.drop(columns=columnas_a_eliminar, inplace=True)
    print("\nColumnas eliminadas con éxito.")
    print("Columnas restantes en el DataFrame:")
    print(df_estaciones_bicimad.columns)
except KeyError as e:
    print(f"\n¡ERROR! Una o más columnas no se encontraron en el DataFrame:")
    print(e)
    # Puedes añadir aquí una lógica para encontrar qué columna exacta falló
    missing_cols = [col for col in columnas_a_eliminar if col not in df_estaciones_bicimad.columns]
    if missing_cols:
        print(f"Las siguientes columnas no se encontraron: {missing_cols}")
except Exception as e:
    print(f"\nOcurrió otro error: {e}")


Columnas eliminadas con éxito.
Columnas restantes en el DataFrame:
Index(['id', 'Número', 'Fecha de Alta', 'Distrito', 'Barrio',
       'Tipo de Reserva', 'Número de Plazas', 'Longitud', 'Latitud',
       'Direccion'],
      dtype='object')


In [8]:
df_estaciones_bicimad.head()

,id,Número,Fecha de Alta,Distrito,Barrio,Tipo de Reserva,Número de Plazas,Longitud,Latitud,Direccion
0,1,001 a,43803,01 CENTRO,01-06 SOL,BiciMAD,30,-3.701998,40.417111,"ALCALA, CALLE, DE, 2"
1,2,001 b,43867,01 CENTRO,01-06 SOL,BiciMAD,30,-3.701564,40.417213,"ALCALA, CALLE, DE, 6"
2,3,2,41813,01 CENTRO,01-05 UNIVERSIDAD,BiciMAD,24,-3.705674,40.420580,"MIGUEL MOYA, CALLE, DE, 1"
3,4,3,41813,07 CHAMBERÍ,07-02 ARAPILES,BiciMAD,18,-3.707212,40.430322,"CONDE DEL VALLE DE SUCHIL, PLAZA, DEL, 2"
4,5,4,41813,01 CENTRO,01-05 UNIVERSIDAD,BiciMAD,24,-3.702674,40.428590,"MANUELA MALASAÑA, CALLE, DE, 3"


Esta parte del codigo toma el archivo de estaciones de bicimad de madrid, y luego busca en las estaciones de metereologia más cernacas asigando las 3 mas cercanas para temperatura y las 2 más cercanas para precipitación

In [9]:
def encontrar_estaciones_meteo_cercanas(bicicleta_coord, df_estaciones_meteo, sensor, n=1):
    """
    Encuentra las estaciones meteorológicas más cercanas *para una estación de bicicleta específica*
    que tengan el sensor especificado.

    Args:
        bicicleta_coord (tuple): Tupla con la latitud y longitud de la estación de bicicleta.
        df_estaciones_meteo (pd.DataFrame): DataFrame de estaciones meteorológicas.
        sensor (str): Columna del sensor a buscar ('T (83)' o 'P (89)').
        n (int): Número de estaciones más cercanas a devolver.

    Returns:
        list: Lista con los códigos de las estaciones meteorológicas más cercanas con el sensor.
    """
    estaciones_con_sensor = df_estaciones_meteo[df_estaciones_meteo[sensor] == 'X'].copy()
    if estaciones_con_sensor.empty:
        return [None] * n

    estaciones_con_sensor['distancia'] = estaciones_con_sensor.apply(
        lambda row: geodesic(bicicleta_coord, (row['LATITUD'], row['LONGITUD'])).km, axis=1
    )
    estaciones_mas_cercanas = estaciones_con_sensor.sort_values(by='distancia').head(n)
    return estaciones_mas_cercanas['CÓDIGO'].tolist()

# Crear columnas para los códigos de las estaciones meteorológicas
df_estaciones_bicimad['E. Temp. Código 1'] = None
df_estaciones_bicimad['E. Temp. Código 2'] = None
df_estaciones_bicimad['E. Temp. Código 3'] = None
df_estaciones_bicimad['E. Prec. Código 1'] = None
df_estaciones_bicimad['E. Prec. Código 2'] = None

# Iterar sobre cada estación de BiciMAD
for index, row_bicimad in df_estaciones_bicimad.iterrows():
    bicicleta_coord = (row_bicimad['Latitud'], row_bicimad['Longitud'])

    # Buscar las tres estaciones meteorológicas más cercanas para temperatura
    codigos_temp = encontrar_estaciones_meteo_cercanas(bicicleta_coord, df_estaciones_meteo, 'T (83)', n=3)
    df_estaciones_bicimad.loc[index, 'E. Temp. Código 1'] = codigos_temp[0]
    df_estaciones_bicimad.loc[index, 'E. Temp. Código 2'] = codigos_temp[1] if len(codigos_temp) > 1 else None
    df_estaciones_bicimad.loc[index, 'E. Temp. Código 3'] = codigos_temp[2] if len(codigos_temp) > 2 else None

    # Buscar las dos estaciones meteorológicas más cercanas para precipitación
    codigos_prec = encontrar_estaciones_meteo_cercanas(bicicleta_coord, df_estaciones_meteo, 'P (89)', n=2)
    df_estaciones_bicimad.loc[index, 'E. Prec. Código 1'] = codigos_prec[0]
    df_estaciones_bicimad.loc[index, 'E. Prec. Código 2'] = codigos_prec[1] if len(codigos_prec) > 1 else None

In [16]:
# Lista de columnas a eliminar (usando los nombres exactos de tu Index)
columnas_a_eliminar = [
   
    "Tipo de Reserva"
]

try:
    # Intenta eliminar las columnas
    df_estaciones_bicimad.drop(columns=columnas_a_eliminar, inplace=True)
    print("\nColumnas eliminadas con éxito.")
    print("Columnas restantes en el DataFrame:")
    print(df_estaciones_bicimad.columns)
except KeyError as e:
    print(f"\n¡ERROR! Una o más columnas no se encontraron en el DataFrame:")
    print(e)
    # Puedes añadir aquí una lógica para encontrar qué columna exacta falló
    missing_cols = [col for col in columnas_a_eliminar if col not in df_estaciones_bicimad.columns]
    if missing_cols:
        print(f"Las siguientes columnas no se encontraron: {missing_cols}")
except Exception as e:
    print(f"\nOcurrió otro error: {e}")


Columnas eliminadas con éxito.
Columnas restantes en el DataFrame:
Index(['id', 'Número', 'Fecha de Alta', 'Distrito', 'Barrio',
       'Número de Plazas', 'Longitud', 'Latitud', 'Direccion',
       'E. Temp. Código 1', 'E. Temp. Código 2', 'E. Temp. Código 3',
       'E. Prec. Código 1', 'E. Prec. Código 2'],
      dtype='object')


In [17]:
df_estaciones_bicimad.head(5)

,id,Número,Fecha de Alta,Distrito,Barrio,Número de Plazas,Longitud,Latitud,Direccion,E. Temp. Código 1,E. Temp. Código 2,E. Temp. Código 3,E. Prec. Código 1,E. Prec. Código 2
0,1,001 a,43803,01 CENTRO,01-06 SOL,30,-3.701998,40.417111,"ALCALA, CALLE, DE, 2",28079035,28079110,28079004,28079056,28079024
1,2,001 b,43867,01 CENTRO,01-06 SOL,30,-3.701564,40.417213,"ALCALA, CALLE, DE, 6",28079035,28079110,28079004,28079056,28079024
2,3,2,41813,01 CENTRO,01-05 UNIVERSIDAD,24,-3.705674,40.420580,"MIGUEL MOYA, CALLE, DE, 1",28079035,28079004,28079110,28079024,28079106
3,4,3,41813,07 CHAMBERÍ,07-02 ARAPILES,18,-3.707212,40.430322,"CONDE DEL VALLE DE SUCHIL, PLAZA, DEL, 2",28079004,28079109,28079035,28079106,28079024
4,5,4,41813,01 CENTRO,01-05 UNIVERSIDAD,24,-3.702674,40.428590,"MANUELA MALASAÑA, CALLE, DE, 3",28079109,28079004,28079035,28079106,28079024


In [25]:
#guardar el resultado en un nuevo archivo CSV
output_file_path = os.path.join(folder_path, 'estaciones_bicimad_meteo_15_05_v1.csv')
df_estaciones_bicimad.to_csv(output_file_path, sep=';', index=False,encoding='utf-8-sig')